# Portfolio Project

We'll perform the following steps:

1. Import Libraries
2. Set Up Directory Paths
3. Prepare Data Generators
4. Build the Custom CNN Model
5. Build the Transfer Learning Model
6. Train Both Models
7. Evaluate and Compare Models

Let's get started!

### 1. Import Libraries

##### Libraries
##### First, import necessary libraries:

In [143]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [145]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [147]:
from sklearn.metrics import classification_report, confusion_matrix

### 2. Set Up Directory Paths
##### Define the paths to your dataset directories and CSV file. Adjust the base_dir to point to where your dataset is located

In [150]:
# Define base directory
base_dir = os.path.join(os.getcwd(), 'multiclass_weather_dataset')

# Directories for training and validation (the 5 class folders)
train_val_dir = base_dir

# Directory and CSV for testing
test_dir = os.path.join(base_dir, 'alien-test')
labels_csv = os.path.join(base_dir, 'test.csv')

testing the dir

In [152]:
# Read the CSV file
labels_df = pd.read_csv(test_csv)

# Verify the DataFrame structure
print("First 5 entries of test.csv:")
print(labels_df.head(10))

# Check column names
print("Columns in test.csv:", labels_df.columns.tolist())


First 5 entries of test.csv:
       Image_id  labels
0   Cloud_1.png       0
1   Cloud_2.jpg       0
2  Cloud_3.jpeg       0
3   Cloud_4.jpg       0
4   foggy_1.jpg       1
5   foggy_2.jpg       1
6   foggy_3.jpg       1
7   foggy_4.jpg       1
8   foggy_5.jpg       1
9   foggy_6.jpg       1
Columns in test.csv: ['Image_id', 'labels']


In [154]:
print("\nUnique labels in test set:", labels_df['labels'].unique())


Unique labels in test set: [0 1 2 3 4]


In [156]:
print(labels_df.head())

       Image_id  labels
0   Cloud_1.png       0
1   Cloud_2.jpg       0
2  Cloud_3.jpeg       0
3   Cloud_4.jpg       0
4   foggy_1.jpg       1


In [158]:
print(labels_df.columns.tolist())

['Image_id', 'labels']


### 3. Prepare Data Generators

##### Training and Validation Generators
Use ImageDataGenerator to create training and validation data generators. We'll use a validation_split of 15%.

In [129]:
# Parameters
img_size = (224, 224)
batch_size = 32

# ImageDataGenerator with data augmentation for training and rescaling for validation
train_val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,  # 15% for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Training generator
train_generator = train_val_datagen.flow_from_directory(
    train_val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation generator
validation_generator = train_val_datagen.flow_from_directory(
    train_val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Print class indices
print("Training Class Indices:", train_generator.class_indices)

Found 1302 images belonging to 6 classes.
Found 228 images belonging to 6 classes.
Training Class Indices: {'alien_test': 0, 'cloudy': 1, 'foggy': 2, 'rainy': 3, 'shine': 4, 'sunrise': 5}


##### Test Generator
Load the test data using flow_from_dataframe with labels from labels.csv.

In [131]:
print(labels_df.columns)

Index(['Image_id', 'labels'], dtype='object')


In [133]:
labels_df['Image_id'] = labels_df['Image_id'].apply(lambda x: os.path.join(test_dir, x))

In [135]:
# Verify the updated DataFrame
print("First 5 entries after updating file paths:")
print(labels_df.head())

First 5 entries after updating file paths:
                                            Image_id  labels
0  C:\Users\Bilal Akhtar\Downloads\Computer Visio...       0
1  C:\Users\Bilal Akhtar\Downloads\Computer Visio...       0
2  C:\Users\Bilal Akhtar\Downloads\Computer Visio...       0
3  C:\Users\Bilal Akhtar\Downloads\Computer Visio...       0
4  C:\Users\Bilal Akhtar\Downloads\Computer Visio...       1


In [137]:
labels_df['labels'] = labels_df['labels'].astype(str)

In [141]:
print("Columns in test.csv:", labels_df.columns.tolist())
print("\nFirst 5 entries in test.csv:")
print(labels_df.head())

Columns in test.csv: ['Image_id', 'labels']

First 5 entries in test.csv:
                                            Image_id labels
0  C:\Users\Bilal Akhtar\Downloads\Computer Visio...      0
1  C:\Users\Bilal Akhtar\Downloads\Computer Visio...      0
2  C:\Users\Bilal Akhtar\Downloads\Computer Visio...      0
3  C:\Users\Bilal Akhtar\Downloads\Computer Visio...      0
4  C:\Users\Bilal Akhtar\Downloads\Computer Visio...      1


In [139]:
# Create the test generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=labels_df,
    x_col='Image_id',
    y_col='labels',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Print class indices
print("Test Class Indices:", test_generator.class_indices)

Found 0 validated image filenames belonging to 0 classes.
Test Class Indices: {}


C:\Users\Bilal Akhtar\anaconda3\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 30 invalid image filename(s) in x_col="Image_id". These filename(s) will be ignored.
  warnings.warn(


### 4. Build the Custom CNN Model
##### Define a simple Convolutional Neural Network from scratch.

In [ ]:
def build_custom_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Get input shape and number of classes
input_shape = (224, 224, 3)
num_classes = train_generator.num_classes

# Build the model
custom_cnn = build_custom_cnn(input_shape, num_classes)
custom_cnn.summary()

#### Transfer Learning Evaluation

In [ ]:
# Evaluate transfer model on validation data
transfer_eval = transfer_model.evaluate(validation_generator)
print(f"Transfer Learning Accuracy: {transfer_eval[1]*100:.2f}%")

# Predictions and classification report
pred_transfer = transfer_model.predict(validation_generator)
pred_transfer_labels = np.argmax(pred_transfer, axis=1)
print(classification_report(validation_generator.classes, pred_transfer_labels, target_names=class_labels))

# Confusion matrix
cm_transfer = confusion_matrix(validation_generator.classes, pred_transfer_labels)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_transfer, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix - Transfer Learning')
plt.show()